In [18]:
import argparse
import os
from typing import Tuple

import gymnasium
import numpy as np
import pandas as pd
import torch
from tianshou.data import Batch, Collector, VectorReplayBuffer
from tianshou.env import DummyVectorEnv
from tianshou.env.pettingzoo_env import PettingZooEnv
from tianshou.policy import BasePolicy, DQNPolicy, MultiAgentPolicyManager
from tianshou.trainer import offpolicy_trainer
from tianshou.utils import TensorboardLogger
from tianshou.utils.net.common import Net
from torch.utils.tensorboard import SummaryWriter

from env.negotiation import NegotiationEnv
from env.negotiation import Outcome
from visualization import AnimatedGraph

In [2]:
def get_parser() -> argparse.ArgumentParser:
    parser = argparse.ArgumentParser()
    parser.add_argument("--seed", type=int, default=1626)
    parser.add_argument("--eps-test", type=float, default=0.05)
    parser.add_argument("--eps-train", type=float, default=0.1)
    parser.add_argument("--buffer-size", type=int, default=20000)
    parser.add_argument("--lr", type=float, default=1e-4)
    parser.add_argument(
        "--gamma", type=float, default=0.9, help="a smaller gamma favors earlier win"
    )
    parser.add_argument("--n-step", type=int, default=3)
    parser.add_argument("--target-update-freq", type=int, default=320)
    parser.add_argument("--epoch", type=int, default=20)
    parser.add_argument("--step-per-epoch", type=int, default=1000)
    parser.add_argument("--step-per-collect", type=int, default=10)
    parser.add_argument("--update-per-step", type=float, default=0.1)
    parser.add_argument("--batch-size", type=int, default=64)
    parser.add_argument(
        "--hidden-sizes", type=int, nargs="*", default=[128, 128, 128, 128]
    )
    parser.add_argument("--training-num", type=int, default=10)
    parser.add_argument("--test-num", type=int, default=10)
    parser.add_argument("--logdir", type=str, default="log")
    parser.add_argument("--render", type=float, default=0.1)
    parser.add_argument(
        "--win-rate",
        type=float,
        default=0.6,
        help="the expected winning rate: Optimal policy can get 0.7",
    )
    parser.add_argument(
        "--watch",
        default=False,
        action="store_true",
        help="no training, " "watch the play of pre-trained models",
    )
    parser.add_argument(
        "--agent-id",
        type=int,
        default=2,
        help="the learned agent plays as the"
        " agent_id-th player. Choices are 1 and 2.",
    )
    parser.add_argument(
        "--resume-path",
        type=str,
        default="",
        help="the path of agent pth file " "for resuming from a pre-trained agent",
    )
    parser.add_argument(
        "--opponent-path",
        type=str,
        default="",
        help="the path of opponent agent pth file "
        "for resuming from a pre-trained agent",
    )
    parser.add_argument(
        "--device", type=str, default="cuda" if torch.cuda.is_available() else "cpu"
    )
    return parser


def get_args() -> argparse.Namespace:
    parser = get_parser()
    return parser.parse_known_args()[0]


def get_agents(
    args: argparse.Namespace = get_args(),
    stakeholder_vals: np.ndarray | None = None
) -> Tuple[BasePolicy, torch.optim.Optimizer, list]:
    env = get_env(stakeholder_vals)
    observation_space = (
        env.observation_space["observation"]
        if isinstance(env.observation_space, gymnasium.spaces.Dict)
        else env.observation_space
    )
    args.state_shape = observation_space.shape or observation_space.n
    args.action_shape = env.action_space.shape or env.action_space.n


    agents = []
    for _ in range(env.env.n_agents):
        net = Net(
            args.state_shape,
            args.action_shape,
            hidden_sizes=args.hidden_sizes,
            device=args.device,
        ).to(args.device)
        optim = torch.optim.Adam(net.parameters(), lr=args.lr)

        agents.append(DQNPolicy(net, optim, args.gamma, args.n_step, target_update_freq=args.target_update_freq))
    policy = MultiAgentPolicyManager(agents, env)
    return policy, env.agents


def get_env(data=None, render_mode=None):
    return PettingZooEnv(NegotiationEnv(stakeholder_matrix=data, render_mode=render_mode))


def train_agent(
    args: argparse.Namespace = get_args(),
    stakeholder_vals: np.ndarray | None = None
) -> Tuple[dict, BasePolicy]:
    # ======== environment setup =========
    train_envs = DummyVectorEnv([lambda: get_env(stakeholder_vals) for _ in range(args.training_num)])
    test_envs = DummyVectorEnv([lambda: get_env(stakeholder_vals) for _ in range(args.test_num)])
    # seed
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    train_envs.seed(args.seed)
    test_envs.seed(args.seed)

    # ======== agent setup =========
    policy, agents = get_agents(args, stakeholder_vals)

    # ======== collector setup =========
    train_collector = Collector(
        policy,
        train_envs,
        VectorReplayBuffer(args.buffer_size, len(train_envs)),
        exploration_noise=True,
    )
    test_collector = Collector(policy, test_envs, exploration_noise=True)
    # policy.set_eps(1)
    train_collector.collect(n_step=args.batch_size * args.training_num)

    # ======== tensorboard logging setup =========
    log_path = os.path.join(args.logdir, "negotiate", "dqn")
    writer = SummaryWriter(log_path)
    writer.add_text("args", str(args))
    logger = TensorboardLogger(writer)

    # ======== callback functions used during training =========
    def save_best_fn(policy):
        if hasattr(args, "model_save_path"):
            model_save_path = args.model_save_path
        else:
            model_save_path = os.path.join(
                args.logdir, "negotiate", "dqn", "policy.pth"
            )
        torch.save(
            policy.policies[agents[args.agent_id - 1]].state_dict(), model_save_path
        )

    def stop_fn(mean_rewards):
        return mean_rewards >= args.win_rate

    def train_fn(epoch, env_step):
        policy.policies[agents[args.agent_id - 1]].set_eps(args.eps_train)

    def test_fn(epoch, env_step):
        policy.policies[agents[args.agent_id - 1]].set_eps(args.eps_test)

    def reward_metric(rews):
        return rews[:, args.agent_id - 1]

    # trainer
    result = offpolicy_trainer(
        policy,
        train_collector,
        test_collector,
        args.epoch,
        args.step_per_epoch,
        args.step_per_collect,
        args.test_num,
        args.batch_size,
        train_fn=train_fn,
        test_fn=test_fn,
        # stop_fn=stop_fn,
        save_best_fn=save_best_fn,
        update_per_step=args.update_per_step,
        logger=logger,
        test_in_train=False,
        reward_metric=reward_metric
    )

    return result, policy.policies

In [50]:
data = pd.read_csv('data/test.csv', header=None).values

args = get_args()
result, policies = train_agent(args, stakeholder_vals=data)

Epoch #1: 1001it [00:01, 636.16it/s, agent_1/loss=19.379, agent_2/loss=35.041, agent_3/loss=70.622, agent_4/loss=4.081, env_step=1000, len=5, n/ep=10, n/st=10, rew=5.74]                          

Epoch #1: test_reward: 5.010000 ± 4.942358, best_reward: 6.090000 ± 4.448696 in #0



Epoch #2: 1001it [00:01, 688.77it/s, agent_1/loss=18.712, agent_2/loss=32.218, agent_3/loss=36.018, agent_4/loss=2.210, env_step=2000, len=5, n/ep=8, n/st=10, rew=8.09]                          

Epoch #2: test_reward: 5.010000 ± 4.942358, best_reward: 6.090000 ± 4.448696 in #0



Epoch #3: 1001it [00:01, 698.22it/s, agent_1/loss=20.772, agent_2/loss=30.352, agent_3/loss=23.611, agent_4/loss=2.234, env_step=3000, len=5, n/ep=7, n/st=10, rew=2.24]                          


...

Epoch #18: test_reward: 4.080000 ± 4.978916, best_reward: 6.090000 ± 4.448696 in #0


Epoch #19: 1001it [00:01, 801.91it/s, agent_1/loss=9.601, agent_2/loss=19.990, agent_3/loss=8.598, agent_4/loss=0.356, env_step=19000, len=7, n/ep=7, n/st=10, rew=5.14]                            


Epoch #19: test_reward: 5.010000 ± 4.942358, best_reward: 6.090000 ± 4.448696 in #0


Epoch #20: 1001it [00:01, 798.52it/s, agent_1/loss=13.011, agent_2/loss=19.146, agent_3/loss=10.222, agent_4/loss=0.352, env_step=20000, len=5, n/ep=5, n/st=10, rew=5.74]                          


Epoch #20: test_reward: 5.010000 ± 4.942358, best_reward: 6.090000 ± 4.448696 in #0


In [51]:
env = get_env(data)
obs, info = env.reset()
done = False
adj_matrices = []
actions = []
while not done:
    agent = env.env.agent_selection
    policy = policies[agent]
    action = policy.forward(batch=Batch(obs=[obs], info=[info])).act[0]
    negotiator = int(agent.split('_')[1])-1
    recipient = action
    adj_matrix = env.env.observe(None)
    np.fill_diagonal(adj_matrix, 0)
    adj_matrices.append(adj_matrix)
    actions.append((negotiator, recipient))
    obs, rew, done, truncated, info = env.step(action)
# Get final state
final_state = env.env.observe(None)
np.fill_diagonal(final_state, 0)
adj_matrices.append(final_state)
actions.append("Final State")
env.close()

In [77]:
animated_graph = AnimatedGraph(adj_matrices, actions, node_labels={0: "Person A", 1: "Person B", 2: "Person C", 3: "Person D"}, interval=1500)
animated_graph.animate()